In [1]:
import lightgbm as lgb
model = lgb.Booster(model_file='model.txt')  # init model

In [2]:
import pandas as pd
df = pd.read_csv('../Data/TTS_2016/Filtered.csv', index_col=0)
df = df[['Vehicles', 'Children', 'Adults', 'Age', 'License', 'Dist', 'PD', 'IncomeClass', 'StudentStatus', 'EmploymentStatus', 'ModeTakenToSchool']]
df

,Vehicles,Children,Adults,Age,License,Dist,PD,IncomeClass,StudentStatus,EmploymentStatus,ModeTakenToSchool
1,2,0,5,19,N,5.801714,6,3,S,O,B
2,0,0,2,22,Y,1.529490,1,3,S,P,W
3,3,1,5,26,Y,32.200970,20,7,S,O,D
4,1,0,4,19,Y,23.502050,16,2,S,O,B
7,1,0,2,31,Y,32.690450,33,7,P,F,9
...,...,...,...,...,...,...,...,...,...,...,...
19840,3,0,5,22,Y,0.000000,2,5,S,F,9
19844,2,0,2,73,Y,23.502260,36,5,P,O,D
19847,0,0,2,43,Y,5.644338,4,4,P,O,9
19852,1,1,2,30,N,11.278420,13,7,P,O,9


In [3]:
df.rename(columns = {'Vehicles': 'Cars', 'Dist': 'Distance', 'License': 'Licence'}, inplace=True)
df['Licence'] = (df['Licence'] == 'Y').astype(int)

df['Income_Lo'] = (df['IncomeClass'] <= 3).astype(int)
df['Income_Hi'] = ((4 <= df['IncomeClass']) & (df['IncomeClass'] <= 6)).astype(int)

df['Status_PT'] = (df['StudentStatus'] == 'P').astype(int)
df['Status_FT'] = (df['StudentStatus'] == 'S').astype(int)

df['Work_PT'] = (df['EmploymentStatus'].isin(list('PJ'))).astype(int)
df['Work_FT'] = (df['EmploymentStatus'].isin(list('FH'))).astype(int)
df['Work_NW'] = (df['EmploymentStatus'].isin(list('O'))).astype(int)

df['Transit'] = (df['ModeTakenToSchool'].isin(list('BGJ'))).astype(int)
df['Active'] = (df['ModeTakenToSchool'].isin(list('CW'))).astype(int)
df['Auto'] = (df['ModeTakenToSchool'].isin(list('DPTU'))).astype(int)

df.drop(columns=['IncomeClass', 'StudentStatus', 'EmploymentStatus', 'ModeTakenToSchool'], inplace=True)
df.describe()

,Cars,Children,Adults,Age,Licence,Distance,PD,Income_Lo,Income_Hi,Status_PT,Status_FT,Work_PT,Work_FT,Work_NW,Transit,Active,Auto
count,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000,15149.000000
mean,1.663608,0.498119,3.095584,26.534887,0.767377,17.587816,20.585121,0.336986,0.482738,0.256057,0.743943,0.345567,0.145158,0.508878,0.340022,0.059344,0.191564
std,1.581760,0.828238,1.218844,10.598385,0.422518,15.259092,14.100439,0.472696,0.499718,0.436468,0.436468,0.475569,0.352272,0.499938,0.473732,0.236275,0.393545
min,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,2.000000,20.000000,1.000000,6.560541,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.000000,3.000000,22.000000,1.000000,13.908040,18.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,2.000000,1.000000,4.000000,29.000000,1.000000,24.218950,35.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,99.000000,6.000000,11.000000,89.000000,1.000000,133.200000,46.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
probs = model.predict(df.values)
preds = [1 if i >= 0.5 else 0 for i in probs]
probs.mean(), sum(preds)/len(preds)

(0.6843334313516645, 0.6968776816951614)

In [5]:
pd.DataFrame({'Family': preds, 'Family_Prob': probs})
df.to_csv('Predictions.csv')